WCET
Interrupts
Get semantic into knuckle


Cryptol examples https://tools.galois.com/cryptol

Low*  

https://link.springer.com/chapter/10.1007/978-3-030-45237-7_6 Highly Automated Formal Proofs over Memory Usage of Assembly Code
https://link.springer.com/10.1007/978-3-319-72308-2_5?fromPaywallRec=false branch free assembly why3


jvm
evm
wasm
might be interesting targets. hmm.


In [14]:
%%file /tmp/hello.c

int main(){
    return 3;
}

Overwriting /tmp/hello.c


In [16]:
!gcc /tmp/hello.c -o /tmp/hello

In [17]:
!objdump -d /tmp/hello | grep main.: -A20

0000000000001129 <main>:
    1129:	f3 0f 1e fa          	endbr64
    112d:	55                   	push   %rbp
    112e:	48 89 e5             	mov    %rsp,%rbp
    1131:	b8 03 00 00 00       	mov    $0x3,%eax
    1136:	5d                   	pop    %rbp
    1137:	c3                   	ret

Disassembly of section .fini:

0000000000001138 <_fini>:
    1138:	f3 0f 1e fa          	endbr64
    113c:	48 83 ec 08          	sub    $0x8,%rsp
    1140:	48 83 c4 08          	add    $0x8,%rsp
    1144:	c3                   	ret


In [18]:
import angr
import archinfo

proj = angr.Project("/tmp/hello")
#arch = archinfo.ArchPcode("x86:LE:64:default")
#proj = angr.Project("/tmp/hello",arch=arch, engine=angr.engines.UberEnginePcode)



There's no tutorial, but it's pretty easy to figure out yourself. There's a bunch of examples in this directory: https://github.com/angr/angr/tree/master/angr/storage/memory_mixins
basically subclass MemoryMixin and write a load and store method
 on slotted_memory.py

In [19]:
from angr.storage.memory_mixins import MemoryMixin

class CustomMemoryMixin(MemoryMixin):
    def load(self, addr, size=None, **kwargs):
        print(f"Custom Load: Address={addr}, Size={size}")
        return super().load(addr, size, **kwargs)

    def store(self, addr, data, **kwargs):
        print(f"Custom Store: Address={addr}, Data={data}")
        super().store(addr, data, **kwargs)
from angr.storage import Memory

class CustomMemory(CustomMemoryMixin, Memory):
    pass
import angr
import archinfo



ImportError: cannot import name 'MemoryMixin' from 'angr.storage.memory_mixins' (/home/philip/philzook58.github.io/.venv/lib/python3.12/site-packages/angr/storage/memory_mixins/__init__.py)

In [57]:
#proj = angr.Project("/tmp/hello")
from angr.storage.memory_mixins import DefaultMemory, AbstractMemory
class CustomMemory():
    def load(self, addr, size=None, **kwargs):
        print(f"Custom Load: Address={addr}, Size={size}")
        return super().load(addr, size, **kwargs)

    def store(self, addr, data, **kwargs):
        print(f"Custom Store: Address={addr}, Data={data}")
        super().store(addr, data, **kwargs)
state = proj.factory.call_state(0x401129, plugins={'memory': DefaultMemory()})
#state.memory = AbstractMemory()
state.block().pp()
print(state.regs.pc)

simgr = proj.factory.simulation_manager(state)
simgr.step(num_inst=2)
simgr.active[0].block().pp()
#simgr.active[0].memory.

#state.memory

SimMemoryError: Trying to allocate stack space in a place that isn't the top of the stack

In [ ]:
state = SimState(arch="x86", mode="symbolic", plugins={"memory": memcls()})

AttributeError: 'CustomMemory' object has no attribute 'SUPPORTS_CONCRETE_LOAD'

In [10]:
import claripy
from angr.storage.memory_mixins.memory_mixin import MemoryMixin
from angr.errors import SimMergeError

class SymbolicArrayMemoryMixin(MemoryMixin):
    """
    A simple example of a memory mixin that uses a single claripy Array
    to represent memory. It loads/stores bytes from/to that Array,
    allowing symbolic addresses without concretization.
    """

    def __init__(self, name='sym_mem', domain_bits=64, data_bits=8, **kwargs):
        super().__init__(**kwargs)
        # Create one Array: domain_bits = 64-bit addresses, data_bits = 8-bit data
        self._domain_bits = domain_bits
        self._data_bits = data_bits
        self._mem = claripy.Array(
            name,
            claripy.BVS('addr_dummy', self._domain_bits).sort(),  # domain sort (BitVec 64)
            claripy.BVS('data_dummy', self._data_bits).sort()     # range sort  (BitVec 8)
        )

    def set_state(self, state):
        super().set_state(state)
        # We'll read the default endianness from the state's arch
        self.endness = state.arch.memory_endness

    def copy(self, memo):
        o = super().copy(memo)
        o._domain_bits = self._domain_bits
        o._data_bits = self._data_bits
        o._mem = self._mem
        return o

    def merge(self, others, merge_conditions, common_ancestor=None):
        """
        Merge is done by creating an if-then-else chain over the entire Array.
        If you have N merges, you end up with nested ITE expressions.

        For example, if we have two memory states M1, M2 and a boolean condition c,
        the new memory after merge is:

            M_merged = If(c, M2, M1)

        In a general multi-way merge we do:
            M_merged = ite_cases(
                [(cond1, M1), (cond2, M2), ...],
                <some-default>
            )
        """
        a = self._mem
        # merge_conditions is something like [True, cond1, cond2, ...] (the first is always True for the base)
        for cond, other in zip(merge_conditions[1:], others):
            # If cond is True, pick other's array, else pick the current a
            a = claripy.If(cond, other._mem, a)

        self._mem = a

        # Return True if you want to indicate that a merge occurred
        return True

    def load(self, addr, size=None, **kwargs):
        """
        Load a bitvector of `size` bytes from memory, starting at `addr`.
        We'll automatically handle endianness. By default, `size` can come from
        the state or from your call site. 
        """
        if size is None:
            size = self.state.arch.bytes  # default to arch word

        endness = kwargs.get('endness', self.endness)

        # Collect each byte from the Array.
        # For big-endian: the byte at addr is the high-order bits in the output.
        # For little-endian: the byte at addr is the low-order bits in the output.
        bytes_ = []
        if endness == 'Iend_BE':
            # For big-endian, the leftmost byte is at the lowest address
            for i in range(size):
                byte_i = claripy.Select(self._mem, addr + i)
                bytes_.append(byte_i)
            result = claripy.Concat(*bytes_)
        else:
            # For little-endian, the leftmost byte is at the highest address in the final bitvector
            # so we read them from high address to low in the final Concat
            for i in reversed(range(size)):
                byte_i = claripy.Select(self._mem, addr + i)
                bytes_.append(byte_i)
            result = claripy.Concat(*bytes_)

        return result

    def store(self, addr, data, size=None, **kwargs):
        """
        Store a bitvector `data` into memory at address `addr`.
        We update self._mem with an If-Then-Else chain if addresses are symbolic,
        but claripy.Store does that internally for us.
        """
        if size is None:
            # data.size() is in bits. So we convert to bytes
            size = data.size() // 8

        endness = kwargs.get('endness', self.endness)

        # We slice out each byte from `data` (which is a bitvector of size*8 bits).
        # For big-endian, the most significant byte is data[size*8-1 : (size-1)*8]
        # For little-endian, the least significant byte is data[7:0], next is data[15:8], etc.
        if endness == 'Iend_BE':
            # The left-most byte in `data` goes to the lowest address
            for i in range(size):
                # The i-th byte from the left
                byte_i = data[((size - i) * 8 - 1) : ((size - i - 1) * 8)]
                self._mem = claripy.Store(self._mem, addr + i, byte_i)
        else:
            # The i-th byte from the right
            for i in range(size):
                byte_i = data[(i+1)*8 - 1 : i*8]
                self._mem = claripy.Store(self._mem, addr + i, byte_i)

    #
    # Additional methods (e.g. changed_bytes, default_value, etc.) can be added
    # but are optional depending on how you want this memory model to behave.
    #


In [13]:
#from angr.storage.memory_mixins.paged_memory.pages import PageMemory

#class MyMemory(PageMemory, SymbolicArrayMemoryMixin):
#    pass

# Then create a state with your memory
proj = angr.Project('/tmp/hello', load_options={'main_opts': {'custom_base_addr': 0}})
st = proj.factory.full_init_state()
st.memory.mem._add_mixin('symbolic_array', SymbolicArrayMemoryMixin())


CRITICAL | 2025-01-22 14:19:32,273 | cle.backends.backend | Deprecation warning: the custom_base_addr parameter has been renamed to base_addr


AttributeError: 'DefaultMemory' object has no attribute 'mem'

In [8]:
state = proj.factory.entry_state()
state.block().pp()

        _start:
401040  endbr64 
401044  xor     ebp, ebp
401046  mov     r9, rdx
401049  pop     rsi
40104a  mov     rdx, rsp
40104d  and     rsp, 0xfffffffffffffff0
401051  push    rax
401052  push    rsp
401053  xor     r8d, r8d
401056  xor     ecx, ecx
401058  lea     rdi, [main]
40105f  call    qword ptr [0x403fd8]


In [40]:
#proj.factory.call_state(0x401129)
#state = proj.factory.full_init_state(addr=0x401129)
def print_state(state):
    print("rip", state.regs.rip)
    print("rsp", state.regs.rsp)
    print("rbp", state.regs.rbp)
    print("rax", state.regs.rax)
    print("rbx", state.regs.rbx)
    print("stack", state.memory.pp())#.load(state.regs.rsp, 0x1, endness='Iend_BE'))
    print(proj.factory.block(state.addr).pp())
    #for reg in state.arch.register_list:
    #    print(f"{reg.name}: {state.regs.__getattr__(reg.name)}")

state = proj.factory.blank_state(addr=0x401129)
#state = proj.factory.full_init_state()


# Ensure the state uses full symbolic execution
# https://github.com/angr/angr/blob/master/angr/sim_options.py
#state.options.discard(angr.options.APPROXIMATE_GUARDS)  # Prevent guard approximation
#state.options.discard(angr.options.SYMBOLIC_WRITE_ADDRESSES)  # Keep symbolic writes
#state.options.discard(angr.options.SYMBOLIC_READ_ADDRESSES)  # Keep symbolic reads
#state.options.discard(angr.options.CONCRETIZE_SYMBOLIC_WRITE_SIZES)  # Prevent concretizing sizes
#state.options.discard(angr.options.CONCRETIZE_SYMBOLIC_READ_SIZES)  # Prevent concretizing read sizes

state.options.add(angr.options.TRACK_MEMORY_ACTIONS)  # Track memory actions
state.options.add(angr.options.STRICT_PAGE_ACCESS)  # Prevent default concretization of memory access
print(type(state.memory))
print_state(state)
simgr = proj.factory.simulation_manager(state)
for i in range(5):
    simgr.step(num_inst=1)
    new_state = simgr.active[0]
    print_state(new_state)



WARNING  | 2025-01-08 18:02:20,579 | angr.storage.memory_mixins.default_filler_mixin | Filling register rbp with 8 unconstrained bytes referenced from 0x401129 (main+0x0 in hello (0x1129))
WARNING  | 2025-01-08 18:02:20,581 | angr.storage.memory_mixins.default_filler_mixin | Filling register rax with 8 unconstrained bytes referenced from 0x401129 (main+0x0 in hello (0x1129))
WARNING  | 2025-01-08 18:02:20,582 | angr.storage.memory_mixins.default_filler_mixin | Filling register rbx with 8 unconstrained bytes referenced from 0x401129 (main+0x0 in hello (0x1129))


<class 'angr.storage.memory_mixins.DefaultMemory'>
rip <BV64 0x401129>
rsp <BV64 0x7ffffffffff0000>
rbp <BV64 reg_rbp_526_64>
rax <BV64 reg_rax_527_64>
rbx <BV64 reg_rbx_528_64>


AttributeError: 'DefaultMemory' object has no attribute 'pp'

In [12]:
ins = proj.factory.block(0x401129, size=4)
ins

<Block for 0x401129, 4 bytes>

In [6]:
state = proj.factory.call_state(0x401129)
#state.step()
b = state.block()


In [46]:
state = proj.factory.call_state(0x401129)
s2 = state.step()
s2, = s2.successors
r = s2.registers
s2.regs.rax

WARNING  | 2025-01-03 09:48:55,915 | angr.storage.memory_mixins.default_filler_mixin | Filling register rbp with 8 unconstrained bytes referenced from 0x40112d (main+0x4 in hello (0x112d))


<BV64 0x3>

AttributeError: 'AngrObjectFactory' object has no attribute 'path'

Right. one of the breakpoints is address concretization. Their memory model is 

https://github.com/angr/angr/blob/master/angr/engines/pcode/emulate.py basic pcode interpreter
https://github.com/angr/angr/blob/master/angr/engines/pcode/behavior.py "behavior". There is a diciontary at the bottom

I can't believe the amount of indirection there is in here.

Ok, but even 

Maybe subclass the pcodenegine mixin.

In [83]:
b = angr.engines.pcode.behavior.BehaviorFactory()
from pypcode import OpCode
import claripy
claripy.backends.z3.convert(b.get_behavior_for_opcode(OpCode.BOOL_AND).evaluate_binary(1,1,claripy.BVV(1,8),claripy.BVV(1,8)))

1

In [81]:
x = claripy.BVS("x", 8) #.dbg_repr()
import claripy.backends.backend_z3
#claripy.backends.backend_z3.BackendZ3().convert(x)
claripy.backends.z3.convert(x)

x_165_8

In [47]:
state.inspect.

In [16]:
main_symbol = proj.loader.main_object.get_symbol("main")
main_symbol

<Symbol "main" in hello at 0x401129>

pywasm https://github.com/mohanson/pywasm

https://github.com/AntonLydike/riscemu

https://github.com/codypierce/pyemu x86
https://github.com/ForceBru/PyVM

https://github.com/mnaberez/py65 6502

In [14]:
import riscemu

from riscemu.riscemu_main import RiscemuMain, RunConfig

# Create an instance of the RiscemuMain class
emulator = RiscemuMain()

# Set the configuration (this is just an example; set according to your needs)
config = RunConfig(stack_size=8192, verbosity=1, flen=64)
emulator.cfg = config

# Set the input files (example assembly files)
emulator.input_files = ["path/to/your/assembly/file.asm"]

# Register all available instruction sets and program loaders
emulator.register_all_isas()
emulator.register_all_program_loaders()

# Call the run method to start the emulator
emulator.run()
cpu = UserModeCPU(self.selected_ins_sets, self.cfg)
cpu.setup_stack(self.cfg.stack_size)
cpu.launch(self.cfg.verbosity > 1)

AssertionError: self.selected_ins_sets cannot be empty